# RAG + RAG Evaluation (LangChain) using Chroma (Python 3.13 friendly) + `OutdoorClothingCatalog_1000.csv` + **gpt-5-mini**

You confirmed these imports work in your Python 3.13 conda env:

```python
from langchain_community.vectorstores import Chroma
import chromadb
```

So this notebook uses **Chroma** instead of FAISS.

It demonstrates:
1. **RAG** over the CSV: load rows → index in Chroma → retrieve top-k → answer with GPT-5 mini using only retrieved context.
2. **RAG groundedness evaluation**: LLM-as-a-judge with a small rubric, returning structured output via `PydanticOutputParser`.

## Assumptions
- `OutdoorClothingCatalog_1000.csv` sits in the **same directory** as this notebook (e.g. `~/opensource/langchain/`).


In [4]:

import os
from typing import List

MODEL = os.environ.get("RAG_DEMO_MODEL", "gpt-5-mini")
USE_OPENAI = bool(os.environ.get("OPENAI_API_KEY"))

CSV_PATH = os.environ.get("RAG_DEMO_CSV_PATH", "OutdoorClothingCatalog_1000.csv")
CHROMA_DIR = os.environ.get("RAG_DEMO_CHROMA_DIR", ".chroma_outdoor_catalog")

print("USE_OPENAI =", USE_OPENAI)
print("MODEL =", MODEL)
print("CSV_PATH =", CSV_PATH)
print("CHROMA_DIR =", CHROMA_DIR)


USE_OPENAI = True
MODEL = gpt-5-mini
CSV_PATH = OutdoorClothingCatalog_1000.csv
CHROMA_DIR = .chroma_outdoor_catalog


## Load the CSV into LangChain `Document`s

Each row becomes a `Document`. For product catalogs this is a good default: one product = one retrievable unit.


In [5]:

import pandas as pd
from langchain_core.documents import Document

df = pd.read_csv(CSV_PATH)
print("Shape:", df.shape)
print("Columns:", list(df.columns))

# We'll use name/description if present; otherwise fall back to "all columns" text.
has_name = "name" in df.columns
has_desc = "description" in df.columns

def row_to_text(row) -> str:
    if has_name or has_desc:
        name = str(row.get("name", "")).strip() if has_name else ""
        desc = str(row.get("description", "")).strip() if has_desc else ""
        if name and desc:
            return f"Product: {name}\nDescription: {desc}"
        if name:
            return f"Product: {name}"
        return desc
    # fallback: stringify all columns
    parts = []
    for col, val in row.items():
        if pd.isna(val):
            continue
        parts.append(f"{col}: {val}")
    return " | ".join(parts)

documents: List[Document] = []
for i, row in df.iterrows():
    text = row_to_text(row)
    meta = {"row_index": int(i)}
    if has_name and pd.notna(row.get("name", None)):
        meta["name"] = str(row["name"])
    documents.append(Document(page_content=text, metadata=meta))

print("Documents:", len(documents))
print("\nExample:\n", documents[0].page_content[:400])
print("Metadata:", documents[0].metadata)


Shape: (1000, 3)
Columns: ['Unnamed: 0', 'name', 'description']
Documents: 1000

Example:
 Product: Women's Campside Oxfords
Description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for 
Metadata: {'row_index': 0, 'name': "Women's Campside Oxfords"}


## Build a Chroma vector store + retriever

- Uses `OpenAIEmbeddings` if `OPENAI_API_KEY` is present.
- Otherwise uses `FakeEmbeddings` so you can still run the pipeline without API calls.

Chroma persists its data to `CHROMA_DIR`.


In [7]:

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import FakeEmbeddings

# Embeddings
if USE_OPENAI:
    from langchain_openai import OpenAIEmbeddings
    embeddings = OpenAIEmbeddings()  # default embedding model
else:
    embeddings = FakeEmbeddings(size=768)

# Build / load Chroma
# Note: if you re-run with different embeddings or docs, consider deleting CHROMA_DIR first.
vs = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=CHROMA_DIR,
)

retriever = vs.as_retriever(search_kwargs={"k": 4})

# Quick check
question = "What are the key features of the EcoFlex 3L Storm Pants?"
hits = retriever.invoke(question)
for i, d in enumerate(hits, 1):
    print(f"[{i}] {d.metadata.get('name','(no name)')}")
    print(d.page_content[:240], "\n")


[1] EcoFlex 3L Storm Pants
Product: EcoFlex 3L Storm Pants
Description: Our new TEK O2 technology makes our four-season waterproof pants even more breathable. It's guaranteed to keep you dry and comfortable – whatever the activity and whatever the weather. Size & Fit 

[2] EcoFlex 3L Storm Pants
Product: EcoFlex 3L Storm Pants
Description: Our new TEK O2 technology makes our four-season waterproof pants even more breathable. It's guaranteed to keep you dry and comfortable – whatever the activity and whatever the weather. Size & Fit 

[3] EcoFlex 3L Storm Pants
Product: EcoFlex 3L Storm Pants
Description: Our new TEK O2 technology makes our four-season waterproof pants even more breathable. It's guaranteed to keep you dry and comfortable – whatever the activity and whatever the weather. Size & Fit 

[4] Women's Waterproof Hiking Trousers
Product: Women's Waterproof Hiking Trousers
Description: Our waterproof rain pants have been redesigned to be tougher and more durable, featuring a lam

## RAG answer generation (GPT-5 mini)

We force the assistant to answer using **only** the retrieved context.


In [8]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

rag_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant.\n"
     "Use ONLY the provided CONTEXT to answer.\n"
     "If the answer is not in the context, say you don't know.\n"),
    ("user", "QUESTION:\n{question}\n\nCONTEXT:\n{context}\n\nANSWER:")
])

def format_context(docs: List[Document]) -> str:
    return "\n\n".join([f"- {d.page_content}" for d in docs])

context_docs = retriever.invoke(question)
context_text = format_context(context_docs)

print("Retrieved context (truncated):\n")
print(context_text[:1200])


Retrieved context (truncated):

- Product: EcoFlex 3L Storm Pants
Description: Our new TEK O2 technology makes our four-season waterproof pants even more breathable. It's guaranteed to keep you dry and comfortable – whatever the activity and whatever the weather. Size & Fit: Slightly Fitted through hip and thigh. 

Why We Love It: Our state-of-the-art TEK O2 technology offers the most breathability we've ever tested. Great as ski pants, they're ideal for a variety of outdoor activities year-round. Plus, they're loaded with features outdoor enthusiasts appreciate, including weather-blocking gaiters and handy side zips. Air In. Water Out. See how our air-permeable TEK O2 technology keeps you dry and comfortable. 

Fabric & Care: 100% nylon, exclusive of trim. Machine wash and dry. 

Additional Features: Three-layer shell delivers waterproof protection. Brand new TEK O2 technology provides enhanced breathability. Interior gaiters keep out rain and snow. Full side zips for easy on/off over

In [9]:

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model=MODEL, temperature=0)
    rag_chain = rag_prompt | llm | StrOutputParser()
    answer = rag_chain.invoke({"question": question, "context": context_text})
else:
    def offline_answer(question: str, context: str) -> str:
        lines = [ln.strip("- ").strip() for ln in context.splitlines() if ln.strip().startswith("-")]
        if not lines:
            return "I don't know based on the provided context."
        return "Based on the retrieved catalog rows: " + " ".join(lines[:2])
    answer = offline_answer(question, context_text)

print("QUESTION:", question)
print("\nANSWER:\n", answer)


QUESTION: What are the key features of the EcoFlex 3L Storm Pants?

ANSWER:
 Key features of the EcoFlex 3L Storm Pants

- TEK O2 technology: air‑permeable membrane for enhanced breathability ("Air In. Water Out.")
- Four‑season waterproof protection via a three‑layer shell
- Slightly fitted through hip and thigh
- Interior/weather‑blocking gaiters to keep out rain and snow
- Full side zips for easy on/off over boots
- Two zippered hand pockets plus a zippered thigh pocket
- Fabric/care: 100% nylon (exclusive of trim); machine wash and dry; imported
- Designed for year‑round outdoor use (including skiing); Official Supplier to the U.S. Ski Team


## Groundedness evaluation (LLM-as-a-judge) with structured output

Rubric:
- `grounded=True` if the answer is supported by the retrieved context.
- `grounded=False` otherwise.


In [10]:

from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Groundedness(BaseModel):
    grounded: bool = Field(..., description="True if the answer is supported by the context.")
    rationale: str = Field(..., description="1-3 sentences explaining why.")

parser = PydanticOutputParser(pydantic_object=Groundedness)

judge_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a judge evaluating whether an ANSWER is supported by the given CONTEXT.\n"
     "If the answer relies on facts not in the context, mark grounded=false.\n"
     "{format_instructions}"),
    ("user", "CONTEXT:\n{context}\n\nANSWER:\n{answer}")
]).partial(format_instructions=parser.get_format_instructions())


In [11]:

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    judge_llm = ChatOpenAI(model=MODEL, temperature=0)
    judge = judge_prompt | judge_llm | parser
    verdict = judge.invoke({"context": context_text, "answer": answer})
else:
    def offline_judge(context: str, answer: str) -> Groundedness:
        ctx = context.lower()
        ans = answer.lower()
        tokens = [t.strip(".,;:!?()[]\"'") for t in ans.split() if len(t) > 4]
        missing = [t for t in tokens if t and t not in ctx]
        grounded = len(missing) < max(10, len(tokens)//2 + 1)
        rationale = (
            "Offline heuristic: most key terms appear in the retrieved context."
            if grounded else
            "Offline heuristic: many key terms do not appear in the retrieved context, suggesting unsupported claims."
        )
        return Groundedness(grounded=grounded, rationale=rationale)

    verdict = offline_judge(context_text, answer)

verdict


Groundedness(grounded=False, rationale="Most bullets match the provided description (TEK O2 breathability, three-layer waterproof shell, fit, gaiters, full side zips, fabric/care, year‑round/ ski use, supplier). However, the answer states the thigh pocket is zippered — the context lists only a 'thigh pocket' (no mention that it is zippered), so that detail is not supported.")

## Batch evaluation (optional)

Run retrieval → answer → judge repeatedly and summarize groundedness.


In [12]:

questions = [
    "What are the key features of the EcoFlex 3L Storm Pants?",
    "Which product mentions sealed seams and waterproofing?",
    "Is there any jacket described as insulated or warm? If so, which one and what does it say?",
]

results = []

for q in questions:
    ctx_docs = retriever.invoke(q)
    ctx = format_context(ctx_docs)

    if USE_OPENAI:
        ans = rag_chain.invoke({"question": q, "context": ctx})
        v = judge.invoke({"context": ctx, "answer": ans})
    else:
        ans = offline_answer(q, ctx)
        v = offline_judge(ctx, ans)

    results.append({
        "question": q,
        "answer": ans,
        "grounded": v.grounded,
        "rationale": v.rationale,
        "top_hits": [d.metadata.get("name", "") for d in ctx_docs],
    })

results


[{'question': 'What are the key features of the EcoFlex 3L Storm Pants?',
  'answer': 'Key features of the EcoFlex 3L Storm Pants\n\n- TEK O2 technology: air‑permeable, brand‑new system that provides enhanced breathability ("Air In. Water Out.").\n- Three‑layer shell for waterproof protection.\n- Interior/weather‑blocking gaiters to keep out rain and snow.\n- Full side zips for easy on/off over boots.\n- Two zippered hand pockets and an additional thigh pocket.\n- Four‑season use — great as ski pants and for year‑round outdoor activities.\n- Fit: slightly fitted through hip and thigh.\n- Fabric & care: 100% nylon (exclusive of trim); machine wash and dry.\n- Imported; official supplier to the U.S. Ski Team.',
  'grounded': True,
  'rationale': "Each listed feature appears explicitly in the provided product descriptions (TEK O2 breathability and 'Air In. Water Out.', three-layer shell, interior gaiters, full side zips, pocket details, four-season/ski use, fit, fabric & care, 'Imported' 